<a href="https://colab.research.google.com/github/aldobc2/Agentes/blob/main/Aspiradoras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 5.0 MB/s 
     |████████████████████████████████| 60 kB 2.8 MB/s 


In [2]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

import random

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [5]:
def obtener_habitacion(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    habitacion = np.zeros((model.grid.width, model.grid.height))
    for celda in model.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
          if isinstance(contenido, GameLifeAgent):
            habitacion[x][y] = 2
          else:
            habitacion[x][y] = contenido.live
    return habitacion

class Piso(Agent):
    def __init__(self, valor):
        self.valor = 1;


class GameLifeAgent(Agent):
    '''
    Representa a un agente o una celda con estado vivo (1) o muerto (0)
    '''
    def __init__(self, unique_id, model):
        '''
        Crea un agente con estado inicial aleatorio de 0 o 1, también se le asigna un identificador 
        formado por una tupla (x,y). También se define un nuevo estado cuyo valor será definido por las 
        reglas mencionadas arriba.
        '''
        super().__init__(unique_id, model)
        self.live = np.random.choice([0,1])
        self.next_state = None

    def step(self):
        neighbours = self.model.grid.get_neighbors( # checamos hacia donde nos podemos mover
            self.pos,
            moore=True,
            include_center=False)
        
    def move(self):
        
        posicionDisp = self.random.choice(neighbours)

        self.model.grid.move_agent(self, posicionDisp)

        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        isFloor = [obj for obj in this_cell if isinstance(obj, Piso)]
        if isFloor:
            if isFloor[0].valor == 1:
                isFloor[1].valor = 0
            else:
              self.step()
    
    def advance(self):
        '''
        Define el nuevo estado calculado del método step.
        '''
        self.live = self.next_state

class GameLifeModel(Model):
    '''
    Define el modelo del juego de la vida.
    '''
    def __init__(self, width, height, cantAgents, avgDirtyCells):
        self.num_agents = cantAgents
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        for (content, x, y) in self.grid.coord_iter():
            Aspiradora = GameLifeAgent((x, y), self)
            self.grid.place_agent(Aspiradora, (1, 1))
            self.schedule.add(Aspiradora)
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": obtener_habitacion})
    
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()

In [6]:
# Definimos el tamaño del Grid
GRID_SIZE_X = 3
GRID_SIZE_Y = 3

# Definimos la cantidad de agentes
cantAgents = 1

# Porcentaje de celdas inicialmente sucias
avgDirtyCells = 0.5

# Tiempo maximo de ejecucion
tiempoMax = 0.6

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
tiempoInicial = str(datetime.timedelta(seconds = tiempoMax))
model = GameLifeModel(GRID_SIZE_X, GRID_SIZE_Y, cantAgents, avgDirtyCells)
for i in range(50):
    model.step()
#while((time.time() - start_time < tiempoMax)):
 # model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.005378


In [7]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(all_grid)



                                                 Grid
0   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
1   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
2   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
3   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
4   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
5   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
6   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
7   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
8   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
9   [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
10  [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
11  [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
12  [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
13  [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
14  [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
15  [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
16  [[0.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, ...
17  [[0.0, 0.0, 0.0], [0.0, 

In [8]:
%%capture

fig, axs = plt.subplots(figsize=(3,3))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap="Greys")

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=50)

In [9]:
anim